In [1]:
from pydantic import BaseModel, ConfigDict, BeforeValidator, Field, EmailStr
from enum import Enum
from typing import Optional, Annotated

PyObjectId = Annotated[str, BeforeValidator(str)]

class SeatType(Enum):
    REGULAR = 0
    PREMIUM = 1
    VIP = 2

class SeatStatus(Enum):
    AVAILABLE = 0
    BOOKED = 1
    RESERVED = 2


class Seat(BaseModel):
    id: Optional[PyObjectId] = Field(alias='_id', default=None) # primary key
    uid: str
    concert_id: str
    venue: str
    seat_number: str
    seat_type: SeatType
    price: float
    status: SeatStatus

    model_config = ConfigDict(use_enum_values=True)


In [ ]:
from mongo import db

In [ ]:
import json

with open('seating_chart.json', 'r') as file:
    data = json.load(file)

del data['schema']
for key, val in data.items():
    print(key, val)

3TEN Austin City Limits Live {'layout': {'GA': [1, 0, 350, 50]}}
Antone's Austin {'layout': {'GA': [1, 0, 750, 15]}}
Austin City Limits Live at The Moody Theater {'layout': {'GA': [1, 0, 300, 35, 50], 'MEZZ': [8, 3, 25, 45], 'BALC': [6, 12, 22, 30]}}
Brauntex Performing Arts Theatre {'layout': {'GA': [1, 0, 586, 40]}}
Cheatham Street Warehouse {'layout': {'GA': [1, 0, 500, 20]}}
Empire - Control Room {'layout': {'GA': [1, 0, 300, 35]}}
Germania Insurance Amphitheater {'layout': {'GA-Pit': [1, 0, 800, 50], '100-Level': [7, 12, 30, 65], '200-Level': [4, 31, 30, 55], 'GA-Lawn': [1, 0, 7000, 20]}}
Gruene Hall {'layout': {'GA': [1, 0, 800, 20]}}
Long Center for the Performing Arts - Dell Hall {'layout': {'Orchestra': [3, 17, 20, 45], 'Parterre': [3, 9, 25, 25]}}
Moody Amphitheater {'layout': {'GA': [1, 0, 5000, 40]}}
Moody Center {'layout': {'Pit': [2, 0, 200, 200], 'GA': [9, 0, 100, 100], '100-Level': [14, 10, 30, 90], '200-Level': [9, 7, 30, 70], 'S-Level': [9, 3, 7, 105], 'P-Level': [9, 

In [4]:
from uuid import uuid4

VIP = ['GA-Pit', 'Orchestra']
PREMIUM = ['100-Level', 'S-Level', 'P-Level']

for venue, info in data.items():
    docs = db['concert'].find({'venue': venue})

    for doc in docs:
        concert_id = doc['uid']
        venue = key
        
        for section, section_data in info['layout'].items():
            num_sections = range(1, section_data[0] + 1)
            num_rows = range(ord('A'), ord(chr(section_data[1] + 66)))
            num_seats = range(1, section_data[2] + 1)
            price_min = section_data[3]

            if section == 'GA':
                for seat in num_seats:
                    seat_number = f'{section} {seat}'
                    seat = Seat(
                        uid=str(uuid4()),
                        concert_id=concert_id,
                        venue=venue,
                        seat_number=seat_number,
                        seat_type=SeatType.REGULAR,
                        price=price_min,
                        status=SeatStatus.AVAILABLE
                    )
                
                    db['seat'].insert_one(seat.__dict__)
            
            else:
                # loop for seat number. maybe zip num_sections, num_rows, and num_seats
                for section_num in num_sections:
                    for row in num_rows:
                        for seat in num_seats:
                            seat_number = f'{section} Section {section_num} Row {chr(row)} Seat {seat}'

                            if section in VIP:
                                seat_type = SeatType.VIP
                            elif section in PREMIUM:
                                seat_type = SeatType.PREMIUM
                            else:
                                seat_type = SeatType.REGULAR

                            seat = Seat(
                                uid=str(uuid4()),
                                concert_id=concert_id,
                                venue=venue,
                                seat_number=seat_number,
                                seat_type=seat_type,
                                price=price_min,
                                status=SeatStatus.AVAILABLE
                            )

                            db['seat'].insert_one(seat.__dict__)

print('Done!')

Done!


In [ ]:
pipeline = [
    {
        '$group': {
            '_id': {
                'concert_id': '$concert_id',
                'venue': '$venue',
                'seat_number': '$seat_number',
                'seat_type': '$seat_type',
                'price': '$price',
                'status': '$status',
                },
            'duplicate_ids': {'$push': '$uid'},
            'count': {'$sum': 1}
        }
    },
    {
        '$match': {
            'count': {'$gt': 1}
        }
    }
]

duplicates = list(db['seat'].aggregate(pipeline))
print(f'{len(duplicates)} duplicates found.')

counter = 0

for doc in duplicates:
        ids_to_remove = doc['duplicate_ids'][1:]  # Keep the first, remove the rest
        if ids_to_remove:
            db['seat'].delete_many({'uid': {'$in': ids_to_remove}})
        counter += 1

print(f"{counter} duplicates documents removed.")

0 duplicates found.
